In [18]:
import polars as pl
import json

import rcs_cataloging as rcc # --> probably need to build first

In [19]:
json_file = '/Users/claysmyth/Desktop/Starr/RCS_Hypno_Data/Benchtop/patient_stim_test/Session1647905860304/DeviceNPC700354H/RawDataTD.json'

In [20]:
with open(json_file, 'r') as file:
    data = json.load(file)

In [21]:
td_data = data[0]['TimeDomainData']

In [22]:
td_data

[{'Header': {'dataSize': 208,
   'dataType': 1,
   'dataTypeSequence': 2,
   'globalSequence': 188,
   'info': 12,
   'systemTick': 16607,
   'timestamp': {'seconds': 696008085},
   'user1': 2,
   'user2': 12},
  'PacketGenTime': -62135568000000,
  'PacketRxUnixTime': 1647905897858,
  'ChannelSamples': [{'Key': 2,
    'Value': [2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,
     2.604317,


In [23]:
td_df = pl.from_dicts(td_data)

SchemaError: unexpected value while building Series of type Float64; found value of type Int64: 0

In [24]:
df = rcc.loop_and_table_td_data(td_data)

In [27]:
df.height

5529